In [1]:
import os 
import warnings
import time
import numpy as np
import scipy.stats
import sys
import sklearn
import sklearn.datasets

from pyspark.sql import SparkSession
warnings.filterwarnings('ignore')
import pandas as pd

# launch this cell if you have issues on windows with py4j (think about updating your PATH)

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# starts a spark session from notebook

os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"


spark = SparkSession \
    .builder \
    .master("yarn") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "8G") \
    .appName("load_explore") \
    .getOrCreate()

sc=spark.sparkContext

22/05/17 19:43:11 WARN Utils: Your hostname, pierre-hp resolves to a loopback address: 127.0.1.1; using 192.168.0.194 instead (on interface eno1)
22/05/17 19:43:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/17 19:43:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Feature Selection

After the feature engineering, we could have a lot of features which are not really worth to give to the predicting model. Considering this problem, we want to use some feature selection algorithms to take the feature which are the most interesting to the model.

We were asked to implement two scalable feature selection algorithms, a ranking algorithm and a forward feature selection.

We could be interested in Minimum-redundancy-maximum-relevance (mRMR) feature selection.


In [ ]:

def generate_dataset(n_samples=100, n_informative=1, n_noisy=2, n_redundant=1, random_seed=0):
    """
    generate a dataset to test
    """
    # Set random seed
    np.random.seed(random_seed)
    
    # Use sklearn.datasets.make_regression to generate an artificial dataset where the output y 
    # is correlated with a subset of of the input features
    X, Y = sklearn.datasets.make_regression(n_samples=n_samples, 
                                            n_features=n_informative+n_noisy, 
                                            n_informative=n_informative)
    
    # Create a random mixing matrix for generating redundant features from informative ones
    mixing_matrix = np.random.random((n_informative, n_redundant))
    
    # Create redundant features by taking random linear combinations of informative features
    redundant_features = np.dot(X[:,0:n_informative], mixing_matrix)
    
    # Add redundant features to the input data
    X = np.concatenate((X, redundant_features), axis=1)
    
    # Return input data X, output data Y
    return X, Y


# Let us generate the dataset
N = 1000
n_informative = 100
n_noisy = 100
n_redundant = 100

X, Y = generate_dataset(n_samples=N, 
                        n_informative=n_informative, 
                        n_noisy=n_noisy, 
                        n_redundant=n_redundant)

t_X = np.transpose(X)

B=3
X_RDD=sc.parallelize(t_X,B).cache()

# Counting the number of rows will allow to implicitly cache the data
X_RDD.count()

In [2]:
def get_score_mrmr(x, y):
    """
    get score of two variable
    """
    
    score = np.abs(scipy.stats.pearsonr(x, y)[0])
    
    return score

In [3]:
# x: Feature to assess
# Broadcasted values:
# broadcast_y: Output feature
# broadcast_selected_features_values: Array of size n_selected_features*N of already selected features
def get_mrmr_score_spark(x):
    """
    method to give score from selected variable
    """
    
    
    # Retrieve content of broadcasted values
    y = broadcast_y.value
    selected_features_values = broadcast_selected_features_values.value
    
    # Get correlation score between feature x and output y (relevance)
    score_x_y_s = get_score_mrmr(x, y)
    
    n_selected_features = selected_features_values.shape[0]
    
    # If some features have already been selected
    if n_selected_features>0:
        
        
        # Get corrrelation scores between x and each feature already selected (redundancy)
        score_features_x_s = np.zeros(n_selected_features, dtype=float)
        
        for j in range(n_selected_features):
                
            score_x_s_j = get_score(x, selected_features_values[j,:])
                
            score_features_x_s[j] = score_x_s_j
                
        # Final score is relevance to output feature - average redundancy with already selected features
        score_x_y_s = score_x_y_s - np.mean(score_features_x_s)
        
    return score_x_y_s

In [4]:
def mrmr_spark(n_total_features, K, sc, X_RDD, Y):
    """
    n_total_features : number of total features
    K : number of feature to select
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution 
    """
    time_execution = []
    
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []

    broadcast_y = sc.broadcast(Y)

    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features_values = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features_values = np.array(selected_features_values)
        
        # Broadcast selected_features_values 
        broadcast_selected_features_values = sc.broadcast(selected_features_values)
    
        # mRMR scores are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `get_mrmr_score_spark` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:get_mrmr_score_spark(x[0])).collect()
    
        # Once all mRMR scores are computed, the index of the feature with the highest score is selected as 
        # in the centralized implementation.
        scores = np.array(scores)
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
    return selected_features_indices, time_execution

In [6]:
n_total_features = t_X.shape[0]
selected_features_indices, execution_time = mrmr_spark(n_total_features, 2, sc,X_RDD, Y)

Step: 0


22/05/17 19:43:17 ERROR Executor: Exception in task 0.0 in stage 4.0 (TID 12)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_14375/755152485.py", line -1, in <lambda>
  File "/tmp/ipykernel_14375/3406059489.py", line 11, in get_mrmr_score_spark
NameError: name 'broadcast_y' is not defined

	at org.apache.

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 12) (192.168.0.194 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(

Then, we will look a the forward feature Selection by training on a decision tree with every feature and add the features which get the best results.

In [7]:
from sklearn import tree
from sklearn.model_selection import train_test_split

def fit_get_score(x):
    """
    train an model to get the score with the addition of a specified x feature
    """
    # Retrieve content of broadcasted values
    y = broadcast_y.value
    selected_features_values = broadcast_selected_features_values.value
    
    
    
    n_selected_features = selected_features_values.shape[0]
    
    if n_selected_features>0:
        # need to merge x and the already selected features
        x = x + selected_features_values
    
    #split data
    x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.3)
    
    # train on training data
    dt = tree.DecisionTreeClassifier()
    dt = dt.fit(x_train,y_train)
    
    # get score on testing set
    score = dt.score(x_test, y_test)
        
        
    return score

In [8]:
def forward_feature_selection(n_total_features, K, sc, X_RDD, Y):
    time_execution = []
    
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    
    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features_values = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features_values = np.array(selected_features_values)
        
        # Broadcast selected_features_values 
        broadcast_selected_features_values = sc.broadcast(selected_features_values)
    
        # mRMR scores are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `fit_get_score` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:fit_get_score(x[0])).collect()
    
        # Once all mRMR scores are computed, the index of the feature with the highest score is selected as 
        # in the centralized implementation.
        scores = np.array(scores)
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
    return selected_features_indices, time_execution
    

In [1]:
spark.stop()

NameError: name 'spark' is not defined

In [9]:
n_total_features = t_X.shape[0]
selected_features_indices, execution_time = forward_feature_selection(n_total_features, 2, sc,X_RDD, Y)

Step: 0


22/05/17 19:43:44 ERROR Executor: Exception in task 2.0 in stage 8.0 (TID 26)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_14375/2130783349.py", line -1, in <lambda>
  File "/tmp/ipykernel_14375/2446780584.py", line 7, in fit_get_score
NameError: name 'broadcast_y' is not defined

	at org.apache.spark.a

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 8.0 failed 1 times, most recent failure: Lost task 2.0 in stage 8.0 (TID 26) (192.168.0.194 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(